# Fairness Experiment — Selector-Based Models (Good vs Bad)

This notebook:

- Loads the dataset from: `../data/synth_data_for_training.csv`
- Defines selectors:
  - **good model** → uses safe prefixes (`valid_prefixes`)
  - **bad model** → uses discriminatory prefixes (`biased_prefixes`)
- Trains both models *without removing columns from the dataframe*
- Wraps models with a `SelectedModel` that applies the selector at prediction time
- Runs ALL partition tests (using full unmodified X_test)
- Runs ALL metamorphic tests (using full unmodified X_test)

Because data is never altered, partitioning and metamorphic tests behave correctly.


In [56]:
import numpy as np
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from partition_tests_2 import PartitionTester
from metamorphic_tests import MetamorphicTester

print("Imports OK.")

Imports OK.


## Load Dataset + Define Feature Selectors

In [57]:
DATA_PATH = "../data/synth_data_for_training.csv"

data = pd.read_csv(DATA_PATH)
y = data['checked']

X_full = data.drop(columns=['checked']).astype(np.float32)

# Allowed (good) prefixes
valid_prefixes = [
    "afspraak_",
    "contacten_soort_",
    "instrument_",
    "deelname_",
    "pla_",
    "typering_",
    "ontheffing_"
]

good_features = [
    col for col in X_full.columns
    if any(col.startswith(p) for p in valid_prefixes)
]

# Biased (bad) prefixes
biased_prefixes = [
    "adres_",
    "persoonlijke_eigenschappen_spreektaal",
    "persoonlijke_eigenschappen_nl_",
    "persoonlijke_eigenschappen_taaleis_",
    "relatie_",
    "belemmering_",
    "beschikbaarheid_",
    "contacten_"
]

biased_features = [
    col for col in X_full.columns
    if any(col.startswith(p) for p in biased_prefixes)
]

print("GOOD model feature count:", len(good_features))
print("BAD model feature count:", len(biased_features))

GOOD model feature count: 132
BAD model feature count: 164


## Train/Test Split
We keep *full* X during splitting.

In [58]:
RANDOM_STATE = 42

X_train, X_test, y_train, y_test = train_test_split(
    X_full, y, test_size=0.25, random_state=RANDOM_STATE, stratify=y
)

print("Train/Test sizes:")
print(X_train.shape, X_test.shape)

Train/Test sizes:
(9483, 315) (3162, 315)


# Partition Tests

In [59]:
partitions = [
    # Gender-based partitions
    {"name": "men", "condition": lambda df: df['persoon_geslacht_vrouw'] == 0},
    {"name": "women", "condition": lambda df: df['persoon_geslacht_vrouw'] == 1},
    # Age-based partitions
    {"name": "young_adults", "condition": lambda df: df['persoon_leeftijd_bij_onderzoek'] < 30},
    {"name": "middle_aged", "condition": lambda df: (df['persoon_leeftijd_bij_onderzoek'] >= 30) & (df['persoon_leeftijd_bij_onderzoek'] < 60)},
    {"name": "seniors", "condition": lambda df: df['persoon_leeftijd_bij_onderzoek'] >= 60},
    # Family status
    {"name": "single_parents", "condition": lambda df: (df['relatie_kind_heeft_kinderen'] == 1) & (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0)},
    {"name": "married_with_children", "condition": lambda df: (df['relatie_kind_heeft_kinderen'] == 1) & (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 1)},
    {"name": "no_children_no_partner", "condition": lambda df: (df['relatie_kind_heeft_kinderen'] == 0) & (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0)},
    # Marital status
    {"name": "currently_married", "condition": lambda df: df['relatie_partner_huidige_partner___partner__gehuwd_'] == 1},
    {"name": "currently_unmarried_with_partner", "condition": lambda df: df['relatie_partner_aantal_partner___partner__ongehuwd_'] > 0},
    {"name": "currently_single", "condition": lambda df: (
        (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0) & 
        (df['relatie_partner_aantal_partner___partner__ongehuwd_'] == 0)
    )},
    {"name": "multiple_unmarried_partners", "condition": lambda df: df['relatie_partner_aantal_partner___partner__ongehuwd_'] > 1},
    {"name": "likely_divorced", "condition": lambda df: (
        (df['relatie_partner_aantal_partner___partner__gehuwd_'] > 0) &  # Had married partner historically
        (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0)  # Not currently married
    )},
    {"name": "likely_divorced_with_children", "condition": lambda df: (
        (df['relatie_partner_aantal_partner___partner__gehuwd_'] > 0) &
        (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0) &
        (df['relatie_kind_heeft_kinderen'] == 1)
    )},
    {"name": "likely_divorced_no_children", "condition": lambda df: (
        (df['relatie_partner_aantal_partner___partner__gehuwd_'] > 0) &
        (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0) &
        (df['relatie_kind_heeft_kinderen'] == 0)
    )},
    {"name": "divorced_women", "condition": lambda df: (
        (df['relatie_partner_aantal_partner___partner__gehuwd_'] > 0) &
        (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0) &
        (df['persoon_geslacht_vrouw'] == 1)
    )},
    {"name": "divorced_women_with_children", "condition": lambda df: (
        (df['relatie_partner_aantal_partner___partner__gehuwd_'] > 0) &
        (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0) &
        (df['persoon_geslacht_vrouw'] == 1) &
        (df['relatie_kind_heeft_kinderen'] == 1)
    )},
    # Currently cohabiting but not married
    {"name": "cohabiting_unmarried", "condition": lambda df: (
        (df['relatie_partner_aantal_partner___partner__ongehuwd_'] > 0) &
        (df['relatie_partner_huidige_partner___partner__gehuwd_'] == 0) &
        (df['relatie_overig_kostendeler'] == 1)  # Cost-sharer = living together
    )},
    # Dutch understanding
    {"name": "understands_dutch", "condition": lambda df: df['persoonlijke_eigenschappen_nl_begrijpen3'] == 1},
    {"name": "does_not_understand_dutch", "condition": lambda df: df['persoonlijke_eigenschappen_nl_begrijpen3'] == 0},
    # Short time at address + language issues (recent immigrants)
    {"name": "likely_recent_arrival_non_Dutch", "condition": lambda df: (
        (df['adres_dagen_op_adres'] < 365) & 
        (df['adres_recentste_plaats_rotterdam'] == 1) &
        (df['persoonlijke_eigenschappen_nl_begrijpen3'] == 0)
    )},
    {"name": "likely_recent_arrival_Dutch", "condition": lambda df: (
        (df['adres_dagen_op_adres'] < 365) & 
        (df['adres_recentste_plaats_rotterdam'] == 1) &
        (df['persoonlijke_eigenschappen_nl_begrijpen3'] == 1)
    )},
    {"name": "less_established_residents_non_Dutch", "condition": lambda df: (
        (df['adres_dagen_op_adres'] < 1825) &
        (df['adres_dagen_op_adres'] >= 365) &
        (df['adres_recentste_plaats_rotterdam'] == 1) &
        (df['persoonlijke_eigenschappen_nl_begrijpen3'] == 0)
    )},
    {"name": "less_established_residents_Dutch", "condition": lambda df: (
        (df['adres_dagen_op_adres'] < 1825) &
        (df['adres_dagen_op_adres'] >= 365) &
        (df['adres_recentste_plaats_rotterdam'] == 1) &
        (df['persoonlijke_eigenschappen_nl_begrijpen3'] == 1)
    )},
    {"name": "established_residents_non_Dutch", "condition": lambda df: (
        (df['adres_dagen_op_adres'] > 1825) &  # 5+ years
        (df['adres_recentste_plaats_rotterdam'] == 1) &
        (df['persoonlijke_eigenschappen_nl_begrijpen3'] == 0)
    )},
    {"name": "established_residents_Dutch", "condition": lambda df: (
        (df['adres_dagen_op_adres'] > 1825) &  # 5+ years
        (df['adres_recentste_plaats_rotterdam'] == 1) &
        (df['persoonlijke_eigenschappen_nl_begrijpen3'] == 1)
    )},
    # Most recent borough
    {"name": "charlois", "condition": lambda df: df['adres_recentste_wijk_charlois'] == 1},
    {"name": "delfshaven", "condition": lambda df: df['adres_recentste_wijk_delfshaven'] == 1},
    {"name": "feijenoord", "condition": lambda df: df['adres_recentste_wijk_feijenoord'] == 1},
    {"name": "ijsselmonde", "condition": lambda df: df['adres_recentste_wijk_ijsselmonde'] == 1},
    {"name": "kralingen_c", "condition": lambda df: df['adres_recentste_wijk_kralingen_c'] == 1},
    {"name": "noord", "condition": lambda df: df['adres_recentste_wijk_noord'] == 1},
    {"name": "prins_alexa", "condition": lambda df: df['adres_recentste_wijk_prins_alexa'] == 1},
    {"name": "stadscentru", "condition": lambda df: df['adres_recentste_wijk_stadscentru'] == 1},
    # Obstacles
    {"name": "psychological_obstacles", "condition": lambda df: df['belemmering_psychische_problemen'] == 1},
    {"name": "no_psychological_obstacles", "condition": lambda df: df['belemmering_psychische_problemen'] == 0},
    {"name": "living_situation_obstacles", "condition": lambda df: df['belemmering_woonsituatie'] == 1},
    {"name": "no_living_situation_obstacles", "condition": lambda df: df['belemmering_woonsituatie'] == 0},
    {"name": "financial_obstacles", "condition": lambda df: df['belemmering_financiele_problemen'] == 1},
    {"name": "no_financial_obstacles", "condition": lambda df: df['belemmering_financiele_problemen'] == 0},
    # Multiple obstacles
    {"name": "psychological_financial_obstacles", "condition": lambda df: (
        (df['belemmering_psychische_problemen'] == 1) & 
        (df['belemmering_financiele_problemen'] == 1)
    )},
    {"name": "psychological_financial_living_obstacles", "condition": lambda df: (
        (df['belemmering_psychische_problemen'] == 1) & 
        (df['belemmering_financiele_problemen'] == 1) &
        (df['belemmering_woonsituatie'] == 1)
    )},
    {"name": "no_obstacles", "condition": lambda df: (
        (df['belemmering_psychische_problemen'] == 0) & 
        (df['belemmering_financiele_problemen'] == 0) &
        (df['belemmering_woonsituatie'] == 0)
    )},
]

In [60]:
def run_partition_tests(X_test, y_test, model, partitions, selected_features):
    """
    Run partition tests with explicit row filtering and column selection.
    """
    print("\n" + "="*70)
    print("PARTITION TEST RESULTS")
    print("="*70)
    
    for partition in partitions:
        try:
            # Step 1: Filter rows based on partition condition
            mask = partition["condition"](X_test)
            X_test_partition = X_test[mask]
            y_test_partition = y_test[mask]
            
            if len(X_test_partition) == 0:
                print(f"\nSkipping {partition['name']}: no rows")
                continue
            
            # Step 2: Select only the allowed columns
            X_test_partition_selected = X_test_partition[selected_features]
            
            # Step 3: Predict
            preds = model.predict(X_test_partition_selected)
            labels = y_test_partition.values
            
            # Step 4: Calculate metrics
            TP = np.sum((preds == 1) & (labels == 1))
            TN = np.sum((preds == 0) & (labels == 0))
            FP = np.sum((preds == 1) & (labels == 0))
            FN = np.sum((preds == 0) & (labels == 1))
            
            N = len(X_test_partition)
            FPR = FP / (FP + TN) if (FP + TN) > 0 else 0
            acc = (TP + TN) / N if N > 0 else 0
            
            print(f"\n{'='*35}")
            print(f"Partition: {partition['name']}")
            print(f"{'='*35}")
            print(f"Rows: {N} | Features: {len(selected_features)}")
            print(f"TP={TP} TN={TN} FP={FP} FN={FN}")
            print(f"Accuracy: {acc*100:.2f}% | FPR: {FPR*100:.2f}%")
            
        except Exception as e:
            print(f"\nError in partition '{partition['name']}': {e}")
            continue
    
    print("\nPartition tests completed.")

# Metamorphic tests

In [61]:
def calculate_violations(name, original_preds, new_preds):
    """
    Calculates how many times the prediction FLIPPED.
    """
    # A violation is when Pred_orig != Pred_new
    violations = np.sum(original_preds != new_preds)
    total = len(original_preds)
    rate = (violations / total) * 100
    
    print(f"Test: {name}")
    print(f"-----------------------------------")
    print(f"Total rows tested: {total}")
    print(f"Prediction Flips:  {violations}")
    print(f"Violation Rate:    {rate:.2f}%")
    print(f"-----------------------------------\n")
    return rate

In [62]:
def test_gender_flip(model, data, selected_features):
    """
    MR: In a fair system, changing gender (Man <-> Woman) should NOT change fraud risk.
    Target Feature: persoon_geslacht_vrouw
    """
    print(">>> Running MR: Gender Flip")
    
    if 'persoon_geslacht_vrouw' not in data.columns:
        print("Skipping: gender column not found")
        return
    
    # 1. Get Baseline Predictions (FIXED: select features BEFORE predicting)
    X_baseline = data[selected_features]
    preds_base = model.predict(X_baseline)
    
    # 2. Apply Transformation
    X_mutant = data.copy()
    X_mutant['persoon_geslacht_vrouw'] = 1 - X_mutant['persoon_geslacht_vrouw']
    
    # 3. Get Mutant Predictions
    X_mutant_selected = X_mutant[selected_features]
    preds_mutant = model.predict(X_mutant_selected)
    
    # 4. Compare
    calculate_violations("Gender Flip", preds_base, preds_mutant)

In [63]:
def test_language_flip(model, data, selected_features):
    """
    MR: In a fair system, understanding Dutch vs Not Understanding 
    should not be the sole decider of fraud.
    """
    target_col = 'persoonlijke_eigenschappen_nl_begrijpen3'
    
    if target_col not in data.columns:
        print(f"Skipping Language Flip: {target_col} not found.")
        return

    print(">>> Running MR: Language Proficiency Flip")
    
    # Baseline (FIXED: select features first)
    X_baseline = data[selected_features]
    preds_base = model.predict(X_baseline)
    
    # Mutant
    X_mutant = data.copy()
    X_mutant[target_col] = 1 - X_mutant[target_col]
    X_mutant_selected = X_mutant[selected_features]
    preds_mutant = model.predict(X_mutant_selected)
    
    calculate_violations("Language Proficiency Flip", preds_base, preds_mutant)

In [64]:
def test_neighborhood_shuffle(model, data, selected_features):
    """
    MR: Changing the neighborhood should not change fraud risk.
    """
    n1 = 'adres_recentste_wijk_feijenoord'
    n2 = 'adres_recentste_wijk_kralingen_c'
    
    if n1 not in data.columns or n2 not in data.columns:
        print("Skipping Neighborhood Shuffle: Columns not found.")
        return

    print(">>> Running MR: Neighborhood Swap (Feijenoord <-> Kralingen)")
    
    # Filter to relevant rows
    mask = (data[n1] == 1) | (data[n2] == 1)
    X_subset = data[mask].copy()
    
    if len(X_subset) == 0:
        print("No relevant rows for neighborhood swap.")
        return

    # Baseline (FIXED: select features first)
    X_baseline = X_subset[selected_features]
    preds_base = model.predict(X_baseline)
    
    # Apply Swap
    X_mutant = X_subset.copy()
    X_mutant[n1] = X_subset[n2]
    X_mutant[n2] = X_subset[n1]
    X_mutant_selected = X_mutant[selected_features]
    preds_mutant = model.predict(X_mutant_selected)
    
    calculate_violations("Neighborhood Swap", preds_base, preds_mutant)

In [65]:
def run_metamorphic_tests(X_test, model, selected_features):
    """Run all metamorphic tests."""
    print("\n" + "="*70)
    print("METAMORPHIC TEST RESULTS")
    print("="*70)
    
    test_gender_flip(model=model, data=X_test, selected_features=selected_features)
    test_language_flip(model=model, data=X_test, selected_features=selected_features)
    test_neighborhood_shuffle(model=model, data=X_test, selected_features=selected_features)
    
    print("Metamorphic tests completed.")

## Train BAD Model (uses biased features)

In [66]:
# bad_base = DecisionTreeClassifier(max_depth=None, min_samples_leaf=1, random_state=RANDOM_STATE)
bad_model = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("gb", GradientBoostingClassifier(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=3,
        random_state=42
    ))
])

bad_model.fit(X_train[biased_features], y_train)
print("Bad model trained.")

Bad model trained.


### Run partition and metamorphic tests

In [67]:
run_partition_tests(X_test, y_test, bad_model, partitions, selected_features=biased_features)


PARTITION TEST RESULTS

Partition: men
Rows: 1660 | Features: 164
TP=23 TN=1490 FP=9 FN=138
Accuracy: 91.14% | FPR: 0.60%

Partition: women
Rows: 1502 | Features: 164
TP=19 TN=1343 FP=4 FN=136
Accuracy: 90.68% | FPR: 0.30%

Partition: young_adults
Rows: 131 | Features: 164
TP=7 TN=95 FP=1 FN=28
Accuracy: 77.86% | FPR: 1.04%

Partition: middle_aged
Rows: 2470 | Features: 164
TP=33 TN=2227 FP=11 FN=199
Accuracy: 91.50% | FPR: 0.49%

Partition: seniors
Rows: 561 | Features: 164
TP=2 TN=511 FP=1 FN=47
Accuracy: 91.44% | FPR: 0.20%

Partition: single_parents
Rows: 1052 | Features: 164
TP=27 TN=896 FP=7 FN=122
Accuracy: 87.74% | FPR: 0.78%

Partition: married_with_children
Rows: 110 | Features: 164
TP=1 TN=98 FP=1 FN=10
Accuracy: 90.00% | FPR: 1.01%

Partition: no_children_no_partner
Rows: 1933 | Features: 164
TP=14 TN=1780 FP=4 FN=135
Accuracy: 92.81% | FPR: 0.22%

Partition: currently_married
Rows: 177 | Features: 164
TP=1 TN=157 FP=2 FN=17
Accuracy: 89.27% | FPR: 1.26%

Partition: curren

In [68]:
run_metamorphic_tests(X_test, bad_model, selected_features=biased_features)


METAMORPHIC TEST RESULTS
>>> Running MR: Gender Flip
Test: Gender Flip
-----------------------------------
Total rows tested: 3162
Prediction Flips:  0
Violation Rate:    0.00%
-----------------------------------

>>> Running MR: Language Proficiency Flip
Test: Language Proficiency Flip
-----------------------------------
Total rows tested: 3162
Prediction Flips:  0
Violation Rate:    0.00%
-----------------------------------

>>> Running MR: Neighborhood Swap (Feijenoord <-> Kralingen)
Test: Neighborhood Swap
-----------------------------------
Total rows tested: 712
Prediction Flips:  0
Violation Rate:    0.00%
-----------------------------------

Metamorphic tests completed.


## Train GOOD Model (uses safe/allowed features)

In [69]:
# good_base = DecisionTreeClassifier(max_depth=None, min_samples_leaf=1, random_state=RANDOM_STATE)
good_model = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("gb", GradientBoostingClassifier(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=3,
        random_state=42
    ))
])

good_model.fit(X_train[good_features], y_train)
print("Good model trained.")

Good model trained.


### Run partition tests

In [72]:
run_partition_tests(X_test, y_test, good_model, partitions, selected_features=good_features)


PARTITION TEST RESULTS

Partition: men
Rows: 1660 | Features: 132
TP=36 TN=1479 FP=20 FN=125
Accuracy: 91.27% | FPR: 1.33%

Partition: women
Rows: 1502 | Features: 132
TP=27 TN=1339 FP=8 FN=128
Accuracy: 90.95% | FPR: 0.59%

Partition: young_adults
Rows: 131 | Features: 132
TP=11 TN=95 FP=1 FN=24
Accuracy: 80.92% | FPR: 1.04%

Partition: middle_aged
Rows: 2470 | Features: 132
TP=49 TN=2213 FP=25 FN=183
Accuracy: 91.58% | FPR: 1.12%

Partition: seniors
Rows: 561 | Features: 132
TP=3 TN=510 FP=2 FN=46
Accuracy: 91.44% | FPR: 0.39%

Partition: single_parents
Rows: 1052 | Features: 132
TP=36 TN=899 FP=4 FN=113
Accuracy: 88.88% | FPR: 0.44%

Partition: married_with_children
Rows: 110 | Features: 132
TP=0 TN=96 FP=3 FN=11
Accuracy: 87.27% | FPR: 3.03%

Partition: no_children_no_partner
Rows: 1933 | Features: 132
TP=25 TN=1763 FP=21 FN=124
Accuracy: 92.50% | FPR: 1.18%

Partition: currently_married
Rows: 177 | Features: 132
TP=2 TN=156 FP=3 FN=16
Accuracy: 89.27% | FPR: 1.89%

Partition: cur

### Run metamorphic tests

In [74]:
run_metamorphic_tests(X_test, good_model, selected_features=good_features)


METAMORPHIC TEST RESULTS
>>> Running MR: Gender Flip
Test: Gender Flip
-----------------------------------
Total rows tested: 3162
Prediction Flips:  0
Violation Rate:    0.00%
-----------------------------------

>>> Running MR: Language Proficiency Flip
Test: Language Proficiency Flip
-----------------------------------
Total rows tested: 3162
Prediction Flips:  0
Violation Rate:    0.00%
-----------------------------------

>>> Running MR: Neighborhood Swap (Feijenoord <-> Kralingen)
Test: Neighborhood Swap
-----------------------------------
Total rows tested: 712
Prediction Flips:  0
Violation Rate:    0.00%
-----------------------------------

Metamorphic tests completed.
